In [7]:

import geopandas
from shapely.geometry import Point
import numpy as np
import pandas as pd
import io
import re
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
pd.options.display.max_rows = 10

In [12]:
neighborhoods = geopandas.read_file('nynta_17d/nynta.shp')
display(neighborhoods)

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK88,Borough Park,39247.228028,5.400502e+07,"POLYGON ((990897.9000244141 169268.1207885742,..."
1,4,Queens,081,QN52,East Flushing,25843.364936,2.945438e+07,"POLYGON ((1041061.948242188 215429.4609985352,..."
2,4,Queens,081,QN48,Auburndale,32446.878673,3.416422e+07,"POLYGON ((1046392.971191406 210169.7536010742,..."
3,4,Queens,081,QN51,Murray Hill,33266.904811,5.248828e+07,"POLYGON ((1038593.459228516 221913.3550415039,..."
4,4,Queens,081,QN27,East Elmhurst,19816.712323,1.972685e+07,"POLYGON ((1022728.275024414 217530.8082275391,..."
...,...,...,...,...,...,...,...,...
190,1,Manhattan,061,MN23,West Village,29400.994067,2.500482e+07,"POLYGON ((987048.2695922852 206851.1467895508,..."
191,2,Bronx,005,BX09,Soundview-Castle Hill-Clason Point-Harding Park,67341.077044,5.198380e+07,(POLYGON ((1017288.198608398 237508.7247924805...
192,2,Bronx,005,BX99,park-cemetery-etc-Bronx,223443.788237,2.208418e+08,(POLYGON ((1020356.162841797 236653.0676269531...
193,4,Queens,081,QN28,Jackson Heights,29422.388817,4.785688e+07,"POLYGON ((1019252.57019043 215025.8591918945, ..."


In [13]:
#Displaying all the boundary conditions
bounds = neighborhoods.bounds
display(bounds)

,minx,miny,maxx,maxy
0,9.822082e+05,162478.111633,9.917731e+05,174107.999390
1,1.033886e+06,211169.669617,1.041164e+06,216737.932434
2,1.040414e+06,208318.344421,1.046393e+06,220023.060791
3,1.032285e+06,215407.440186,1.042341e+06,223347.718994
4,1.018490e+06,214963.454590,1.024560e+06,220477.852600
...,...,...,...,...
190,9.802614e+05,203099.065796,9.870579e+05,209759.140991
191,1.016247e+06,232396.226196,1.029605e+06,241334.257385
192,1.009255e+06,234392.177795,1.048780e+06,271349.052185
193,1.011993e+06,211378.829590,1.020429e+06,220392.760986


In [14]:
# Map crimes to neighborhoods
def find_neighborhood(x_coord, y_coord):
    '''
    Takes a pair of X and Y coordinates (using the NAD_1983_StatePlane_New_York_Long_Island coordinate system)
    and returns the corresponding New York City neighborhood.
    '''
    if np.isnan(x_coord) or np.isnan(y_coord):
        return np.nan
    
    point = Point(x_coord, y_coord)
    matches = neighborhoods.geometry.contains(Point(x_coord, y_coord))
    
    if any(matches):
        return neighborhoods.loc[matches, 'NTAName'].values[0]
    
    return np.nan

In [19]:
print(find_neighborhood(1004528,195225))

Bushwick North


In [ ]:
years = range(2006, 2016)

for crime_type in ['felony', 'misdemeanor', 'violation']:
    for year in years:
        # Define filename
        folder = dataloc + '/bk_slice_' + crime_type + '/'
        filename = folder + crime_type + '_{}_desc_year_borough'.format(year)
        
#         # Add filenames here if you'd like to skip any
#         if filename in []:
#             continue
        
        print('Beginning filename '+filename) # Just to keep track of progresss
        
        # Define regular expressions so that we can correctly parse columns with commas in them
        latlong_regex = re.compile(r'(\([^\(\)]+\))')
        unclassified_regex = re.compile(r',([^0-9]*,UNCLASSIFIE),')
        gambling_regex = re.compile(r',(LOITERING\/GAMBLING \(.*),BROOKLYN')
        
        # Replace parentheses with quotes and pipe into file-like object
        # Adapted from an answer to this StackOverflow question:
        # http://stackoverflow.com/questions/35948417/how-to-use-square-brackets-as-a-quote-character-in-pandas-read-csv
        with io.open(filename+'.out', 'rb') as fin:
            fo = io.StringIO()
            fo.writelines(unicode(re.sub(gambling_regex,
                                         r',"\1",BROOKLYN',
                                         re.sub(unclassified_regex,
                                                r',"\1",',
                                                re.sub(latlong_regex, r'"\1"', line))))
                                  for line in fin)
            fo.seek(0)

        # Read transformed CSV into dataframe
        crimes = pd.read_csv(fo, index_col=None, header=None, quotechar='"')

        # Drop any rows that are missing x or y coordinates
        crimes = crimes[~pd.isnull(crimes[6])]
        crimes = crimes[~pd.isnull(crimes[7])]
        
        # Find the neighborhood for each crime
        crimes.insert(len(crimes.columns.values), len(crimes.columns.values),
              crimes.apply(lambda row: find_neighborhood(row[6], row[7]), axis=1))
        
        crimes.to_csv(filename+'_with_hood.out', index=False, header=False)